<a href="https://colab.research.google.com/github/renatojmf/Aspects-Extraction-in-Portuguese/blob/main/CAt_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-Processamento

In [1]:
! pip install stanza

     |████████████████████████████████| 432 kB 4.8 MB/s 
     |████████████████████████████████| 175 kB 44.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=e955320d35a681322bfae401f00ac8bcc65879ddd3e50849bd7b36c0c34e4a3e
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import stanza as st
import pandas as pd
st.download('pt') # download pt model
nlp = st.Pipeline('pt') # initialize pt neural pipeline

In [ ]:
! pip install unidecode

In [3]:
df = pd.read_csv('/content/restaurantes.csv',sep=';')

#passar apenas os textos/frases para a função
def getCoNLLU(df):
  dicts = []
  for texto in df:
    doc = nlp(texto) # doc is class Document
    dicts.append(doc.to_dict())

  return dicts

In [ ]:
from unidecode import unidecode
import re

words = []

for word in df['text']:
  old_string = unidecode(word)
  new_string = re.sub(r"[^a-zA-Z0-9]"," ",old_string)
  words.append(new_string.lower())

In [ ]:
df['text'] = words

In [ ]:
df.head()

,text,aspects,aspectsClass
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food
1,excelente peixe para quem esta localizado no ...,peixe,Food
2,serve lanches e cerveja gelada tem mesinhas f...,cerveja,Drinks
3,muito bom o atendimento,atendimento,Service
4,cardapio variado,Cardápio,Food


In [ ]:
dicts = getCoNLLU(df['text'])

saida = ''
for sentenca in dicts[0:2]:
  for tokens in sentenca:
    for tks in tokens:
      for tag in tks:
        #print("tag:" ,tag)
        saida += str(tks[tag])  + "\t"
      saida +="\n"
    saida +="\n"
  saida +="\n"


print(saida)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


1	hamburgueria	hamburgueria	NOUN	Gender=Fem|Number=Sing	11	nsubj	0	12	
2	artesanal	artesanal	ADJ	Gender=Fem|Number=Sing	1	amod	13	22	
3	com	com	ADP	4	case	23	26	
4	tempero	tempero	NOUN	Gender=Masc|Number=Sing	1	nmod	27	34	
5	diferenciado	diferenciar	VERB	Gender=Masc|Number=Sing|VerbForm=Part	4	acl	35	47	
6	de	de	ADP	8	case	49	51	
7	primeira	primeiro	ADJ	Gender=Fem|NumType=Ord|Number=Sing	8	amod	52	60	
8	qualidade	qualidade	NOUN	Gender=Fem|Number=Sing	4	nmod	61	70	
9	fora	ser	AUX	Mood=Ind|Number=Sing|Person=3|Tense=Pqp|VerbForm=Fin	11	cop	72	76	
10	o	o	DET	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	11	det	77	78	
11	atendimento	atendimento	NOUN	Gender=Masc|Number=Sing	0	root	79	90	
(12, 13)	no	91	93	
12	em	em	ADP	14	case	
13	o	o	DET	Gender=Masc|Number=Sing|PronType=Art	14	det	
14	qual	qual	PRON	Gender=Masc|Number=Sing|PronType=Rel	15	obl	94	98	
15	diferencia	diferenciar	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	11	acl:relcl	99	109	
16	por	por	SCONJ	18	mark	110	113

In [ ]:
def getNouns():
  for sentenca in dicts:
    for tokens in sentenca:
      for tks in tokens:
        for tag in tks:
          if tks[tag] == 'NOUN':
            yield (tks['text'].lower())
          

In [ ]:
from collections import Counter, defaultdict
import json

c = Counter()
c.update(getNouns())
nouns = getNouns()
print(c)

json.dump(dict(c), open('/content/nouns.json', 'w'))

Counter({'atendimento': 74, 'comida': 52, 'ambiente': 38, 'lugar': 33, 'preco': 31, 'pratos': 23, 'restaurante': 23, 'pena': 16, 'local': 15, 'carne': 15, 'qualidade': 14, 'opcao': 14, 'pessoas': 13, 'servico': 12, 'variedade': 12, 'rodizio': 11, 'visita': 9, 'conta': 9, 'peixe': 8, 'casa': 8, 'cardapio': 8, 'massa': 8, 'carnes': 8, 'almoco': 7, 'prato': 7, 'tambaqui': 7, 'garcons': 7, 'estacionamento': 7, 'tempero': 6, 'reais': 6, 'musica': 6, 'manaus': 6, 'bebidas': 6, 'problema': 6, 'localizacao': 6, 'pizza': 5, 'garcom': 5, 'regiao': 5, 'peixes': 5, 'sucos': 5, 'buffet': 5, 'carte': 5, 'delicia': 5, 'valor': 5, 'precos': 5, 'cervejas': 4, 'vezes': 4, 'pizzas': 4, 'acompanhamento': 4, 'arroz': 4, 'vivo': 4, 'camarao': 4, 'entrada': 4, 'resultado': 4, 'batida': 4, 'cidade': 4, 'experiencia': 4, 'acustica': 4, 'jantar': 4, 'mundo': 4, 'padrao': 4, 'pais': 4, 'salgados': 4, 'opcoes': 4, 'empanadas': 3, 'vinho': 3, 'decoracao': 3, 'beira': 3, 'cozida': 3, 'brocolis': 3, 'coco': 3, 'cupu

In [ ]:
def getCorpus2(df):
  corpus = []
  for item in df:
    corpus.append(item.split())
  
  return corpus

In [ ]:
from gensim.models import Word2Vec
corpus = getCorpus2(df['text'])
#print(corpus[1::])

f = Word2Vec(corpus[1::],
                 sg=0,
                 negative=5,
                 window=10,
                 size=200,
                 min_count=2,
                 iter=5,
                 workers=10)

f.wv.save_word2vec_format("/content/my_word_vectors.vec")


In [ ]:
d = json.load(open('/content/nouns.json'))
nouns2 = Counter()
#print(f.wv.vocab.items[0])
for k, v in d.items():
  for tag,_ in f.wv.vocab.items():
    if k.lower() == tag:
      #print("valor v: ",v)
      nouns2[k.lower()] += v

# nouns2, _ = zip(*sorted(nouns2.items(),
#                 key=lambda x: x[1],
#                 reverse=True))

# json.dump(nouns, open("/content/aspect_words.json", "w"))

In [ ]:
nouns3, _ = zip(*sorted(nouns2.items(),
                key=lambda x: x[1],
                reverse=True))

json.dump(nouns3, open("/content/aspect_words.json", "w"))

# Métricas e Funções do algoritmo

In [ ]:
! pip install reach

In [40]:
import json
from reach import Reach
from collections import defaultdict
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

In [6]:
def normalize(x):
    """Normalize a vector while controlling for zero vectors."""
    x = np.copy(x)
    if np.ndim(x) == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / np.linalg.norm(x)
    norm = np.linalg.norm(x, axis=-1)
    mask = norm > 0
    x[mask] /= norm[mask][:, None]
    return x


In [7]:
"""Simple method."""
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter


def get_aspects(fragments, embeddings, n_adj_seed, n_nouns, min_count):
    """Get aspects based on fragments."""
    adj, _, noun = zip(*fragments)
    adj_cand, _ = zip(*Counter(adj).most_common(n_adj_seed))

    cands = candidate(embeddings,
                      adj,
                      noun,
                      adj_cand,
                      n_nouns,
                      min_count)

    return cands


def candidate(embeddings,
              adj,
              noun,
              seed_words,
              n_nouns,
              min_count):
    """
    Generates candidate aspects based on adjective co-occurrences

    Parameters
    ----------
    embeddings : Reach
        A Reach instance containing the word embeddings.
    constructions : list of tuples
        A list of adjective noun tuples.
    seed_words : list of str
        A list of strings. All these words should be in vocab for the
        given embeddings model.
    frequency_threshold : int
        Any noun occurring fewer times than this threshold is discarded
    n_nouns : int
        The amount of items to return

    Returns
    -------
    candidates : dict
        A dictionary mapping strings to their scores.

    """
    a = list(set(adj))
    sims = embeddings.similarity(a, seed_words).max(1)
    adj_scores = dict(zip(a, sims))

    noun_scores = defaultdict(lambda: [0, 0])
    for adj, noun in zip(adj, noun):
        noun_scores[noun][0] += adj_scores[adj]
        noun_scores[noun][1] += 1

    noun_scores = {k: v[0] for k, v in noun_scores.items()
                   if v[1] > min_count}

    return sorted(noun_scores.items(), key=lambda x: x[1])[-n_nouns:]


def rbf_attention(vec, memory, gamma, **kwargs):
    """
    Single-head attention using RBF kernel.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items
    gamma : float
        the gamma of the RBF kernel.

    Returns
    -------
    attention : np.array
        A (1, N)-shaped array, representing a single-headed attention mechanism

    """
    z = rbf_kernel(vec, memory, gamma)
    s = z.sum()
    if s == 0:
        # If s happens to be 0, back off to uniform
        return np.ones((1, len(vec))) / len(vec)
    return (z.sum(1) / s)[None, :]


def softmax(x, axis=1):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis, keepdims=True))
    s = e_x.sum(axis=axis, keepdims=True)
    return e_x / s


def attention(vec, memory, **kwargs):
    """
    Standard multi-head attention mechanism.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items

    Returns
    -------
    attention : np.array
        A (M, N)-shaped array, representing the attention over all memories.

    """
    z = memory.dot(vec.T)
    return softmax(z)


def mean(vec, aspect_vecs, **kwargs):
    """Just a mean weighting."""
    return (np.ones(len(vec)) / len(vec))[None, :]

In [64]:
def get_scores(instances,
               aspects,
               r,
               labels,
               remove_oov=False,
               attention_func=attention,
               **kwargs):
    """Scoring function."""
    #print(labels)
    assert all([x in r.items for x in labels])
    
    label_vecs = normalize(r.vectorize(labels))
    aspect_vecs = [x.mean(0)
                   for x in r.transform(aspects,
                                        remove_oov=False)]
    aspect_vecs = np.stack(aspect_vecs)
    if len(instances) == 1:
        instances = [instances]

    t = r.transform(instances, remove_oov=remove_oov)

    out = []
    for vec in t:
        att = attention_func(vec, aspect_vecs, **kwargs)
        #print("att",att)
        # Att = (n_heads, n_words)
        z = att.dot(vec)
        # z = (n_heads, n_dim)
        x = normalize(z).dot(label_vecs.T)
        # x = (n_heads, n_labels)
        out.append(x.sum(0))
    return np.stack(out)

# Avaliação com o dataset restaurantes.csv

## Processamento e escolhas de labels para os dados

In [15]:
df['labels'] =-1
df.head()

,text,aspects,aspectsClass,labels
0,Hamburgueria artesanal com tempero diferenciad...,tempero,Food,-1
1,"Excelente peixe, para quem esta localizado no ...",peixe,Food,-1
2,"Serve lanches e cerveja gelada, tem mesinhas f...",cerveja,Drinks,-1
3,Muito bom o atendimento.,atendimento,Service,-1
4,Cardápio variado.,Cardápio,Food,-1


In [ ]:
df['aspectsClass'].unique()

array(['Food', 'Drinks', 'Service', 'Ambience', ' Price', 'Location',
       'Price', 'Miscellaneous', 'General', nan, 'service'], dtype=object)

In [18]:
df.loc[df["aspectsClass"] == "Service", "labels"] = 0
df.loc[df["aspectsClass"] == "Food", "labels"] = 1
df.loc[df["aspectsClass"] == "Price", "labels"] = 2
df.loc[df["aspectsClass"] == "Ambience", "labels"] = 3
df.loc[df["aspectsClass"] == "Location", "labels"] = 4

In [36]:
df_tst = df[df['labels'] != -1]
df_tst.head()

,text,aspects,aspectsClass,labels
0,Hamburgueria artesanal com tempero diferenciad...,tempero,Food,1
1,"Excelente peixe, para quem esta localizado no ...",peixe,Food,1
3,Muito bom o atendimento.,atendimento,Service,0
4,Cardápio variado.,Cardápio,Food,1
5,Ambiente agradável.,Ambiente,Ambience,3


In [60]:
y_true = df_tst['labels']

## Avaliações com word2vec cbow_s50

In [35]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/cbow_s50.txt",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
              "o atendimento estava incrível, recomendo demais".lower().split(), 
              "um ambiente de otima qualidade".lower().split(),
              "o lugar é muito bem localizado".lower().split(),
              "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
print(instances)
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
#[i.lower().split() for i in instances]
s = get_scores(instances,
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

pred = s.argmax(1)
#print(s)
print(pred)

[['a', 'comida', 'estava', 'deliciosa', 'e', 'muito', 'bem', 'temperada'], ['o', 'atendimento', 'estava', 'incrível,', 'recomendo', 'demais'], ['um', 'ambiente', 'de', 'otima', 'qualidade'], ['o', 'lugar', 'é', 'muito', 'bem', 'localizado'], ['achei', 'o', 'preco', 'um', 'pouco', 'caro', 'mas', 'vale', 'a', 'pena']]
[1 0 3 4 1]


In [61]:
from sklearn.metrics import f1_score

GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/cbow_s50.txt",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = df_tst['text']
# instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
#               "o atendimento estava incrível, recomendo demais".lower().split(), 
#               "um ambiente de otima qualidade".lower().split(),
#               "o lugar é muito bem localizado".lower().split(),
#               "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
#print(instances)
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
#[i.lower().split() for i in instances]
s = get_scores([i.lower().split() for i in instances],
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

y_pred = s.argmax(1)
#print(s)
print(y_pred)

[1 4 1 2 3 1 1 1 1 4 1 1 1 1 1 1 3 1 1 1 1 0 1 1 1 1 0 0 1 1 4 4 4 4 1 1 1
 1 1 1 1 4 1 1 1 1 1 1 0 1 0 1 1 1 1 3 3 0 0 2 3 3 1 3 1 1 1 3 3 3 1 2 1 0
 1 1 1 1 0 1 1 1 3 0 4 1 1 1 1 3 3 0 0 3 3 3 3 1 3 0 3 1 2 1 1 1 2 1 1 1 1
 3 3 3 3 1 1 3 1 1 1 0 1 1 0 1 1 3 3 4 1 4 0 3 1 1 1 1 4 1 3 0 1 1 1 1 1 1
 1 3 1 1 1 1 1 1 4 1 0 0 0 3 3 3 3 1 1 1 1 1 1 1 3 3 3 1 1 3 1 3 1 1 1 1 1
 3 1 1 1 0 1 1 1 1 1 3 1 0 1 1 1]


In [62]:
f1_score(y_true, y_pred, average='macro')

0.3747126436781609

In [63]:
print(classification_report(y_true, y_pred,target_names=label_set))

              precision    recall  f1-score   support

 atendimento       0.59      0.30      0.40        43
      comida       0.56      0.83      0.67        82
       preco       0.60      0.16      0.25        19
    ambiente       0.35      0.35      0.35        40
       lugar       0.25      0.18      0.21        17

    accuracy                           0.50       201
   macro avg       0.47      0.36      0.37       201
weighted avg       0.50      0.50      0.47       201



In [43]:
f1_score = precision_recall_fscore_support(y_true, y_pred)
f1_macro = precision_recall_fscore_support(y_true,y_pred,average="weighted")
print(f1_score)
print()
print(f1_macro)

(array([0.59090909, 0.55737705, 0.6       , 0.35      , 0.25      ]), array([0.30232558, 0.82926829, 0.15789474, 0.35      , 0.17647059]), array([0.4       , 0.66666667, 0.25      , 0.35      , 0.20689655]), array([43, 82, 19, 40, 17]))

(0.5013134773227751, 0.5024875621890548, 0.4683279007262538, None)


## Avaliação com my_word_vectors gerado no pré-processamento do dataset

In [45]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = ["a comida estava deliciosa e muito bem temperada".lower().split(), 
              "o atendimento estava incrível, recomendo demais".lower().split(), 
              "um ambiente de otima qualidade".lower().split(),
              "o lugar é muito bem localizado".lower().split(),
              "achei o preco um pouco caro mas vale a pena".lower().split()]
#instances = df['text'][5:10]
print(instances)
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
#[i.lower().split() for i in instances]
s = get_scores(instances,
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

pred = s.argmax(1)
print(s)
print(pred)

[['a', 'comida', 'estava', 'deliciosa', 'e', 'muito', 'bem', 'temperada'], ['o', 'atendimento', 'estava', 'incrível,', 'recomendo', 'demais'], ['um', 'ambiente', 'de', 'otima', 'qualidade'], ['o', 'lugar', 'é', 'muito', 'bem', 'localizado'], ['achei', 'o', 'preco', 'um', 'pouco', 'caro', 'mas', 'vale', 'a', 'pena']]
[[0.9547266  0.95819604 0.9406847  0.9333397  0.94423556]
 [0.9700805  0.94707143 0.9397762  0.93460137 0.94361514]
 [0.943849   0.9407705  0.94308615 0.9519595  0.94289553]
 [0.94415426 0.9342752  0.94304526 0.9306154  0.9592436 ]
 [0.9499382  0.9430925  0.9535146  0.9381563  0.9412436 ]]
[1 0 3 4 2]


In [54]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]
instances = df_tst['text']
label_set = ['atendimento', 'comida', 'preco', 'ambiente', 'lugar']
s = get_scores([i.lower().split() for i in instances],
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

y_pred = s.argmax(1)
print(y_pred)

[0 0 0 0 3 0 0 0 4 4 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 2 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 4 4 1 3 0 0 1 3 3 0 0 0 1 0
 0 0 0 1 0 2 1 1 3 0 0 1 0 0 3 4 4 0 0 1 4 4 4 1 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 4 0 3 0 0 0 0 0 1 0 0 0 0 0 4 0 4 0 2 0 0 0 0 4 0 3 0 1 1 1 1 0 0
 0 3 0 0 0 0 0 0 4 4 0 0 0 0 0 0 0 0 0 1 1 0 0 2 0 0 0 0 0 0 0 0 4 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0]


In [55]:
f1_score(y_true, y_pred, average='macro')

0.25541954017822244

In [59]:
print(classification_report(y_true, y_pred,target_names=label_set))

              precision    recall  f1-score   support

 atendimento       0.26      0.84      0.40        43
      comida       0.65      0.24      0.35        82
       preco       0.20      0.05      0.08        19
    ambiente       0.89      0.20      0.33        40
       lugar       0.12      0.12      0.12        17

    accuracy                           0.33       201
   macro avg       0.42      0.29      0.26       201
weighted avg       0.52      0.33      0.31       201



In [56]:
f1_score = precision_recall_fscore_support(y_true, y_pred)
f1_macro = precision_recall_fscore_support(y_true,y_pred,average="weighted")
print(f1_score)
print()
print(f1_macro)

(array([0.25899281, 0.64516129, 0.2       , 0.88888889, 0.11764706]), array([0.8372093 , 0.24390244, 0.05263158, 0.2       , 0.11764706]), array([0.3956044 , 0.3539823 , 0.08333333, 0.32653061, 0.11764706]), array([43, 82, 19, 40, 17]))

(0.5243555821367621, 0.3333333333333333, 0.31185122142629174, None)


# Funções auxiliares

In [ ]:
with open('/content/restaurant_textos.txt ', 'w') as textfile:
  textfile.write("\n".join(str(item) for item in sentences))
textfile.close()

In [ ]:
with open('/content/corpus.txt ', 'w') as textfile:
  for item in corpus:
    textfile.write(str(item))
textfile.close()

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def colorize(words, color_array):
    cmap=matplotlib.cm.Blues
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color))
        print(color)
        colored_string += template.format(color, '&nbsp' + word + '&nbsp')
    return colored_string

words = instances[0]#['um', 'ambiente', 'de', 'ótima', 'qualidade']
color_array = atts[0][0]

print(color_array)
s = colorize(words, color_array)

# or simply save in an html file and open in browser
with open('/content/colorize.html', 'w') as f:
    f.write(s)

[0.1934354  0.19726068 0.19610389 0.20825753 0.20494252]
#d1e2f3
#d0e2f2
#d0e2f2
#cee0f2
#cfe1f2
